#### Train an MLP with 2 inputs, 3-4+ hidden units and one output on the following examples (XOR function):


Step 1) Build a perceptron
A stack of perceptrons together == hidden layer a.k.a a dense layer

In [83]:
import numpy as np
import pandas as pd
np.random.seed(42)

In [116]:
class MLP:
    def __init__(self, inputs, outputs, hidden_units=3, epochs=20, learning_rate=.1, random_state=None):
        self.inputs = inputs
        self.outputs = outputs
        self.hidden_units = hidden_units
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.random_state = random_state
    def checkParameters(self):
        if self.inputs < 1:
            raise ValueError('Must have at least 1 input')
        if self.outputs < 1:
            raise ValueError('Must have at least 1 output')
        if self.hidden_units < 1:
            raise ValueError('Must have at least 1 hidden unit')
        if self.epochs < 1:
            raise ValueError('Must train for at least 1 epoch')
        #Is it a max of 1 also??
        if self.learning_rate <=0:
            raise ValueError('Learning rate must be greater than 0')
    
    def fit(self, x, y):
        self.checkParameters()
        self.input = x
        self.output = y
        if self.random_state is not None:
            np.random.seed(self.random_state)
        #Xavier initialization to train model to converge faster
        self.w1 = np.random.normal(loc=0.0,
                              scale=np.sqrt(2/(self.inputs+self.hidden_units)),
                             size = (self.inputs, self.hidden_units))
        self.w2 = np.random.normal(loc=0.0,
                              scale=np.sqrt(2/(self.hidden_units+self.outputs)),
                             size = (self.hidden_units, self.outputs))
        self.h1 = np.zeros(self.hidden_units)
        ## bias can be added later if necessary
        
        self.forward()
        self.backward(y)
    def predict(self, x):
        pass
    
    def myLogLoss(self, y):
        return -((y * np.log(self.output)) + (1 - y)  * np.log(1 - self.output))
    
    def myCost(self, loss):
        return np.mean(loss)

    def randomise(self):
        pass
    
    def sigmoid(self, x):
        return (1 / (1 + np.exp(-x)))
    
    def forward(self):
        z1 = np.dot(self.input, self.w1)
        self.hidden = self.sigmoid(z1)
        z2 = np.dot(self.hidden, self.w2)
        self.output = self.sigmoid(z2)
      #  print(self.output)
        
        
    
    def backward(self, y):
        print(self.output)
        loss = (self.myLogLoss(y))
        w1 = w1 - (self.learning_rate * ()
        print(loss)
        print(self.myCost(loss))

In [117]:
x = np.array([0, 1])
y = np.array([1, 0])
test = MLP(2,2,3,random_state=0)
test.fit(x,y)

[0.61434556 0.60915575]
[0.48719771 0.93944613]
0.7133219229438384
